In [28]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [176]:
with open('data.txt', 'r') as f:
    data = f.readlines()

# inputs
# d/st rankings (1-32) best = 1, worst = 32
xs = [int(i) for i in data[0].split(',')]   
# targets
# fantasy points scored by RBs against each d/st
ys = [float(i) for i in data[1].split(',')]

# Training data
defense_ranks = np.array(xs[:10], dtype=np.float32)
fantasy_scores = np.array(ys[:10], dtype=np.float32)

# Testing data
test_defense_ranks = np.array(xs[10:], dtype=np.float32)
test_fantasy_scores = np.array(ys[10:], dtype=np.float32)

# Convert data to PyTorch tensors
inputs = torch.from_numpy(defense_ranks).view(-1, 1)
targets = torch.from_numpy(fantasy_scores).view(-1, 1)
test_inputs = torch.from_numpy(test_defense_ranks).view(-1, 1)
test_targets = torch.from_numpy(test_fantasy_scores).view(-1, 1)

# # One-hot encode the defense ranks
# num_ranks = 32  # Assuming 32 possible ranks
# inputs = F.one_hot(torch.from_numpy(defense_ranks), num_classes=num_ranks).float()
# test_inputs = F.one_hot(torch.from_numpy(test_defense_ranks), num_classes=num_ranks).float()

In [447]:
# Define the neural network architecture
class RegressionModel(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(RegressionModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, 1)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu(out)
        out = self.fc3(out)
        return out

# Set hyperparameters
input_size = 1
hidden_size = 20
learning_rate = 0.001
num_epochs = 500

# Create the model
model = RegressionModel(input_size, hidden_size)

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

In [457]:
# Training loop
for epoch in range(num_epochs):
    # Forward pass
    outputs = model(inputs)
    loss = criterion(outputs, targets)

    # Backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Print the loss for every epoch
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}")

# Make predictions
with torch.no_grad():
    predicted = model(test_inputs)

# Print the predictions and original scores
for i in range(len(test_defense_ranks)):
    print(f"Defense Rank: {test_defense_ranks[i]}, Predicted Fantasy Score: {predicted[i][0]}, Actual Fantasy Score: {test_fantasy_scores[i]}")

Epoch 1/500, Loss: 12.54907512664795
Epoch 2/500, Loss: 11.090994834899902
Epoch 3/500, Loss: 12.536815643310547
Epoch 4/500, Loss: 11.07919979095459
Epoch 5/500, Loss: 12.524473190307617
Epoch 6/500, Loss: 11.06778335571289
Epoch 7/500, Loss: 12.512046813964844
Epoch 8/500, Loss: 11.05667781829834
Epoch 9/500, Loss: 12.499552726745605
Epoch 10/500, Loss: 11.045823097229004
Epoch 11/500, Loss: 12.487014770507812
Epoch 12/500, Loss: 11.035192489624023
Epoch 13/500, Loss: 12.474424362182617
Epoch 14/500, Loss: 11.024734497070312
Epoch 15/500, Loss: 12.461797714233398
Epoch 16/500, Loss: 11.014422416687012
Epoch 17/500, Loss: 12.449138641357422
Epoch 18/500, Loss: 11.004217147827148
Epoch 19/500, Loss: 12.436437606811523
Epoch 20/500, Loss: 10.994117736816406
Epoch 21/500, Loss: 12.423712730407715
Epoch 22/500, Loss: 10.984090805053711
Epoch 23/500, Loss: 12.410968780517578
Epoch 24/500, Loss: 10.974130630493164
Epoch 25/500, Loss: 12.398185729980469
Epoch 26/500, Loss: 10.964215278625488